In [7]:
# Importing data analytics related libraries (DataFrame and Visualization)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

# Adjusting the size of matplotlib
import matplotlib as mpl
mpl.rc('figure', figsize=(8, 7))
mpl.__version__

# Adjusting the style of matplotlib
style.use('ggplot')

# Magic to tell python notebook we want matplotlib charts included
%matplotlib inline

# importing pulp as an optimization solver
from pulp import *
import re

import matplotlib.pyplot as plt
from IPython.display import Image

In [8]:
!pip install pulp

In [9]:
Download=pd.read_csv("DKSalaries-4.csv")

In [10]:
prob = pulp.LpProblem('', pulp.LpMaximize)

AttributeError: module 'pulp.pulp' has no attribute 'LpMaximize'

In [ ]:
In [15]:
#create decision variables
decision_variables = []
for rownum, row in dfclean.iterrows():
    variablestr = str('x' + str(rownum)) #Create naming of variables
    variable = pulp.LpVariable(str(variablestr), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable) #Appending variables

print ("Total number of decision_variables: " + str(len(decision_variables)))

In [ ]:
#create optimization function
total_books = ""
for rownum, row in dfclean.iterrows():
    for i, book in enumerate(decision_variables):
        if rownum == i:
            total_books += row['rating.1']*book
            
prob += total_books
print ("Optimization function: "+str(total_books))

In [ ]:

#create constrains - there are only 365 days
hours_week_read = 5
pages_per_hour = 60

total_pages_needs_to_read = ""
for rownum, row in dfclean.iterrows():
    for i, schedule in enumerate(decision_variables):
        if rownum == i:
            formula = row['pages']*schedule
            total_pages_needs_to_read += formula
# 50% of my book is fantasy
            
total_pages_can_read = 52*hours_week_read*pages_per_hour

prob += (total_pages_needs_to_read <= total_pages_can_read)

In [ ]:
print prob
prob.writeLP("BuyingBestsellers.lp" )

In [ ]:
optimization_result = prob.solve()
assert optimization_result == pulp.LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [ ]:
#reorder results
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index(by='variable')

#append results
for rownum, row in dfclean.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            dfclean.loc[rownum, 'decision'] = results_row['value']